# Backpropagation y SGD: Desmitificando el Corazón de las Redes Neuronales

¡Hola a todos! Es un placer acompañarlos en este camino de aprendizaje. El día de hoy, por encargo del **Dr. Fuentes**, exploraremos dos conceptos fundamentales que permiten que las máquinas "aprendan": el **Backpropagation** (Propagación hacia atrás) y el **SGD** (Descenso de Gradiente Estocástico).

--- 

## 1. La Intuición Detrás del Aprendizaje

Imaginen que están en la cima de una montaña neblinosa y quieren llegar al valle más bajo. No pueden ver el camino completo, pero pueden sentir la inclinación del suelo bajo sus pies. 

*   **La pendiente (Gradiente):** Nos dice hacia dónde subir. Si queremos bajar, debemos ir en dirección opuesta.
*   **El paso (Learning Rate):** Qué tan grande es el paso que damos en esa dirección.

En una red neuronal, la "montaña" es nuestra **Función de Pérdida** ($L$) y el suelo sobre el que caminamos está definido por los **Pesos** ($w$) y **Sesgos** ($b$) de la red.

## 2. Descenso de Gradiente Estocástico (SGD)

El Descenso de Gradiente tradicional usa *todo* el conjunto de datos para calcular un solo paso. ¡Eso es muy lento si tenemos millones de datos! 

El **SGD** simplifica esto: en cada paso, toma **un solo ejemplo** (o un pequeño grupo llamado *mini-batch*) al azar. Aunque el camino sea un poco más "ruidoso", llega al objetivo mucho más rápido.

### La ecuación matemática fundamental:
$$\theta_{t+1} = \theta_t - \eta \cdot \nabla_{\theta} J(\theta; x^{(i)}, y^{(i)})$$

Donde:
*   $\theta$: Los parámetros (pesos).
*   $\eta$: La tasa de aprendizaje (Learning Rate).
*   $\nabla_{\theta} J$: El gradiente de la función de costo respecto a los parámetros.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

# Configuración estética
plt.style.use('ggplot')
%matplotlib inline

### Visualización Interactiva del SGD
Observemos cómo la tasa de aprendizaje afecta nuestra búsqueda del mínimo en una función de juguete: $f(x) = x^2$.

In [ ]:
def visualize_sgd(lr, steps):
    x = np.linspace(-10, 10, 100)
    y = x**2
    
    # Simulación de pasos
    current_x = 9.0  # Empezamos en un punto lejano
    history_x = [current_x]
    history_y = [current_x**2]
    
    for _ in range(steps):
        gradient = 2 * current_x # Derivada de x^2
        current_x = current_x - lr * gradient
        history_x.append(current_x)
        history_y.append(current_x**2)
        
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, 'b-', alpha=0.5, label='Función de Pérdida $x^2$')
    plt.plot(history_x, history_y, 'ro-', label='Pasos del SGD')
    plt.title(f'SGD con Learning Rate: {lr}')
    plt.xlabel('Peso (w)')
    plt.ylabel('Pérdida (L)')
    plt.legend()
    plt.show()

interact(visualize_sgd, 
         lr=FloatSlider(min=0.01, max=1.1, step=0.05, value=0.1, description='Learn Rate'), 
         steps=IntSlider(min=1, max=50, step=1, value=10, description='Pasos'))

interactive(children=(FloatSlider(value=0.1, description='Learn Rate', max=1.1, min=0.01, step=0.05), IntSlide…

<function __main__.visualize_sgd(lr, steps)>

## 3. Backpropagation: La Regla de la Cadena

Si el SGD nos dice *cómo movernos*, el Backpropagation nos dice *cuánto contribuyó cada peso* al error final. Es simplemente una aplicación recursiva de la **Regla de la Cadena** de cálculo.

Supongamos una red diminuta:
$z = w \cdot x + b\\$
$a = \sigma(z)\\$ (Función de activación)
$\\L = (a - y)^2\\$ (Error cuadrático)

Para ajustar $w$, necesitamos $\frac{\partial L}{\partial w}$:
$$\frac{\partial L}{\partial w} = \frac{\partial L}{\partial a} \cdot \frac{\partial a}{\partial z} \cdot \frac{\partial z}{\partial w}$$

Esto es como desarmar una cebolla por capas, desde el resultado final hacia atrás.

### Ejemplo de Juguete: Aprendiendo una suma simple
Vamos a crear una neurona que aprenda que $2x = y$. El peso ideal debería ser 2.

In [ ]:
def solve_toy_problem(epochs, lr):
    # Datos: x=1, y=2; x=2, y=4; x=3, y=6
    X = np.array([1, 2, 3, 4, 5])
    Y = np.array([2, 4, 6, 8, 10])
    
    w = 0.0 # Inicialización
    losses = []
    weights = []
    
    for epoch in range(epochs):
        # Elegimos un punto al azar (SGD puro)
        idx = np.random.randint(0, len(X))
        x, y_true = X[idx], Y[idx]
        
        # 1. Forward Pass
        y_pred = w * x
        
        # 2. Computar Pérdida (MSE parcial)
        loss = (y_pred - y_true)**2
        losses.append(loss)
        weights.append(w)
        
        # 3. Backpropagation (Gradiente)
        # dL/dw = dL/dy_pred * dy_pred/dw
        # dL/dy_pred = 2 * (y_pred - y_true)
        # dy_pred/dw = x
        grad = 2 * (y_pred - y_true) * x
        
        # 4. Actualizar peso
        w = w - lr * grad
        
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    ax1.plot(losses)
    ax1.set_title('Pérdida por interacción')
    ax1.set_xlabel('Iteración')
    
    ax2.plot(weights)
    ax2.axhline(y=2.0, color='r', linestyle='--')
    ax2.set_title(f'Evolución del Peso (Final: {w:.2f})')
    ax2.set_xlabel('Iteración')
    plt.show()

interact(solve_toy_problem, 
         epochs=IntSlider(min=10, max=500, step=10, value=100), 
         lr=FloatSlider(min=0.001, max=0.1, step=0.005, value=0.01, readout_format='.3f'))

interactive(children=(IntSlider(value=100, description='epochs', max=500, min=10, step=10), FloatSlider(value=…

<function __main__.solve_toy_problem(epochs, lr)>

---
### Conclusiones para llevar a casa:
1.  **Backpropagation** es el algoritmo para calcular gradientes de forma eficiente.
2.  **SGD** es la estrategia de optimización que usa esos gradientes para actualizar los pesos.
3.  La **Tasa de Aprendizaje** es el hiperparámetro más crítico: muy alto y divergemos (saltamos el valle), muy bajo y nunca llegamos.

¡Espero que esto les sea de gran utilidad en sus estudios! Quedo a su disposición para cualquier duda.

Con cariño,
**Su asistente IA** (con el apoyo del Dr. Fuentes)